# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tshikapa,-6.4167,20.8000,23.22,93,78,1.87,CD,1707240325
1,1,key west,24.5557,-81.7826,19.00,77,100,9.26,US,1707240231
2,2,punta arenas,-53.1500,-70.9167,18.01,55,0,5.66,CL,1707240124
3,3,iwanai,42.9744,140.5089,-6.69,95,96,1.82,JP,1707240326
4,4,edinburgh of the seven seas,-37.0676,-12.3116,16.85,69,8,2.94,SH,1707240327


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal = city_data_df.loc[city_data_df["Humidity"] < 50] 
city_ideal = city_ideal.loc[city_ideal["Cloudiness"] == 0]

# Drop any rows with null values
city_ideal.dropna(how='any')

# Display sample data
city_ideal.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,rikaze,29.2500,88.8833,-2.51,16,0,0.77,CN,1707240338
38,38,san miguel,13.4833,-88.1833,32.81,34,0,6.17,SV,1707240343
50,50,ushuaia,-54.8000,-68.3000,17.81,36,0,3.60,AR,1707240248
71,71,chai badan,15.2000,101.1333,26.74,41,0,2.76,TH,1707240360
113,113,mount pleasant,32.7941,-79.8626,13.50,47,0,7.72,US,1707240381


### Step 3: Create a new DataFrame called `hotel_df`.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,rikaze,CN,29.2500,88.8833,16,
1,san miguel,SV,13.4833,-88.1833,34,
2,ushuaia,AR,-54.8000,-68.3000,36,
3,chai badan,TH,15.2000,101.1333,41,
4,mount pleasant,US,32.7941,-79.8626,47,
5,beeville,US,28.4008,-97.7483,42,
6,aswan,EG,24.0934,32.9070,26,
7,saraipali,IN,21.3333,83.0000,45,
8,hafar al-batin,SA,28.4328,45.9708,27,
9,yanbu,SA,24.0891,38.0637,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rikaze - nearest hotel: Gesar Hotel
san miguel - nearest hotel: Continental Plaza Hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
chai badan - nearest hotel: Narai Grand Hotel
mount pleasant - nearest hotel: Old Village Post House
beeville - nearest hotel: La Quinta by Wyndham
aswan - nearest hotel: Yaseen hotel
saraipali - nearest hotel: No hotel found
hafar al-batin - nearest hotel: فندق الفرحان
yanbu - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
etzatlan - nearest hotel: Hotel Cuatro Caminos
bafoulabe - nearest hotel: No hotel found
gaoual - nearest hotel: Hotel
bamako - nearest hotel: Grand Hôtel
bhum - nearest hotel: No hotel found
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
tidjikja - nearest hotel: No hotel found
qaisumah - nearest hotel: No hotel found
knoxville - nearest hotel: Hilton Knoxville
alice - nearest hotel: No hotel found
villa luvianos - nearest hotel: No hotel foun

,City,Country,Lat,Lng,Humidity,Hotel Name
0,rikaze,CN,29.2500,88.8833,16,Gesar Hotel
1,san miguel,SV,13.4833,-88.1833,34,Continental Plaza Hotel
2,ushuaia,AR,-54.8000,-68.3000,36,Apart Hotel Aires del Beagle
3,chai badan,TH,15.2000,101.1333,41,Narai Grand Hotel
4,mount pleasant,US,32.7941,-79.8626,47,Old Village Post House
5,beeville,US,28.4008,-97.7483,42,La Quinta by Wyndham
6,aswan,EG,24.0934,32.9070,26,Yaseen hotel
7,saraipali,IN,21.3333,83.0000,45,No hotel found
8,hafar al-batin,SA,28.4328,45.9708,27,فندق الفرحان
9,yanbu,SA,24.0891,38.0637,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=["City", "Country"],
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "Hotel Name"
)

# Display the mapmap_plot

map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)